# Step 2 - Weather Data Process - Get Weather Station Data for Mapped Weather Stations

<b>Summary:</b><br>
Uses Synoptic Weather Data API to grab weather station data for the mapped weather stations to cameras.<br>
Synoptic is partner of SDG&E, helping store and serve their weather station data--accessible via API.

- Read in processed camera weather station mappings
- Get all station data for input timerange
- Save weater station data by network

<b>Output:</b><br>
.<br>
├── data<br>
&emsp;&emsp;&emsp;└── raw<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;├── weather_HPWREN.csv<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;├── weather_SC-EDISON.csv<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;└── weather_SDGE.csv

<b>Instructions:</b><br>
- Create a copy of `config.json.example` and remove the `.example`
- Update the `synoptic_api_token` value

In [1]:
import json
import os
from ast import literal_eval
from io import StringIO

import fastparquet
import geopandas
import matplotlib.pyplot as plt
import pandas as pd
import requests
import rtree
from geopandas import GeoDataFrame
from shapely.geometry import Point, Polygon
from tqdm.notebook import tqdm, trange

## Parameters

In [2]:
# Base url and API token
with open("config.json") as config_file:
    cfg = json.load(config_file)

API_BASE_URL = "https://api.synopticdata.com/v2/"
API_TOKEN = cfg["synoptic_api_token"]

# should be SHORTNAME values from mesonet - should align to notebook 1
WEATHER_NETWORKS = {139: "SDGE", 81: "HPWREN", 231: "SC-EDISON"}

# figlib start = 2016-06-04 (assume PST)
# figlib end = 2021-07-11 (assume PST)
# YYYYmmddHHMM (format needed for api)
START_DATETIME = "201606030000"
END_DATETIME = "202107120000"
# END_DATETIME = "202112311159"

## 1. Get data for matched weather stations

### Read weather station mappings

In [29]:
station_mappings_df = pd.read_csv("../../data/processed/camera_station_mappings.csv")
print(station_mappings_df.shape)

(309, 7)


In [30]:
station_mappings_df

,properties.description.id,stid,shortname,distance_m,distance_mi,is_in_direction,rn
0,hpwren0_unknown direction,BFDSD,SDGE,2362.963219,1.468277,NaN,1
1,hpwren0_unknown direction,CVXSD,SDGE,8161.908209,5.071575,NaN,2
2,hpwren0_unknown direction,DJZSD,SDGE,10216.081526,6.347979,NaN,3
3,hpwren1_north,HP016,HPWREN,0.000000,0.000000,True,1
4,hpwren1_north,MGDSD,SDGE,4637.139293,2.881385,True,2
...,...,...,...,...,...,...,...
304,hpwren30_south,TLGSD,SDGE,4521.722898,2.809668,True,2
305,hpwren30_south,CRISD,SDGE,8167.055431,5.074773,True,3
306,hpwren30_west,HP024,HPWREN,0.000000,0.000000,True,1
307,hpwren30_west,TLGSD,SDGE,4521.722898,2.809668,True,2


### Helper function to get weather station data given stid and start/end time

In [31]:
def get_historical_station_readings(stid: str, start: str, end: str) -> str:
    """
    Return csv string of station readings given station id(s) and timeframe.
    Start and end format = YYYYmmddHHMM.

    Note:
    For multiple stids, would need to remove output csv, but would be in json format.
    """
    # All times are requested in UTC, but may be returned in either UTC or Local time
    endpoint = f"{API_BASE_URL}stations/timeseries"
    params = {
        "token": API_TOKEN,
        "stid": stid,
        "start": start,
        "end": end,
        "obtimezone": "UTC",
        "output": "csv",
    }
    r = requests.get(endpoint, params=params)
    if r.status_code not in range(200, 299):
        # TODO: add error handling
        print("Error!")
        return {}
    # return r.json()
    return r.text

### Get all data for time range given station network shortname

In [33]:
def get_network_station_data(df: pd.DataFrame, shortname: str) -> pd.DataFrame:
    df = df[df["shortname"] == shortname]
    stations = df["stid"].dropna().unique().tolist()

    weather_df_created = 0

    for idx, station in enumerate(tqdm(stations)):
        # print(station)
        # get csv string
        csv_string_io = StringIO(
            get_historical_station_readings(
                stid=station,
                start=START_DATETIME,
                end=END_DATETIME
                # start="201606030000",
                # end="201606030100",
            )
        )
        # initial creation of df
        if not weather_df_created:
            weather_df = pd.read_csv(csv_string_io, skiprows=6, header=[0, 1])
            weather_df_created = 1
        # additional appends to df
        else:
            temp_df = pd.read_csv(csv_string_io, skiprows=6, header=[0, 1])
            weather_df = pd.concat([weather_df, temp_df])

    return weather_df

In [34]:
for key in WEATHER_NETWORKS:
    # if key == 139:
    #     continue
    # print(WEATHER_NETWORKS[key])
    network_station_df = get_network_station_data(
        station_mappings_df, WEATHER_NETWORKS[key]
    )
    network_station_df.to_csv(
        f"../../data/raw/weather_{WEATHER_NETWORKS[key]}.csv", index=False
    )
    # break

  0%|          | 0/87 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

In [35]:
# %%time

# weather_df_created = 0

# for idx, station in enumerate(tqdm(stations)):
#     # print(station)
#     # get csv string
#     csv_string_io = StringIO(
#         get_historical_station_readings(
#             stid=station,
#             start=START_DATETIME,
#             end=END_DATETIME
#             # start="201606030000",
#             # end="201606030100",
#         )
#     )
#     # initial creation of df
#     if not weather_df_created:
#         weather_df = pd.read_csv(csv_string_io, skiprows=6, header=[0, 1])
#         weather_df_created = 1
#     # additional appends to df
#     else:
#         temp_df = pd.read_csv(csv_string_io, skiprows=6, header=[0, 1])
#         weather_df = pd.concat([weather_df, temp_df])

# # prev runtime: 1h 6min 56s

In [36]:
# weather_df.shape

In [37]:
# weather_df.to_csv("../../data/raw/weather.csv", index=False)